In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)


In [2]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

Suppose the Fashion MNIST dataset only contained eight
classes—for example, all the classes except for sandal and shirt. Someone built and
trained a Keras model on that set and got reasonably good performance (>90% accuracy).
Let’s call this model A. You now want to tackle a different task: you have images
of sandals and shirts, and you want to train a binary classifier (positive=shirt,
negative=sandal).

## Reusing a Keras model

Let's split the fashion MNIST training set in two:

- X_train_A: all images of all items except for sandals and shirts (classes 5 and 6).
- X_train_B: a much smaller training set of just the first 200 images of sandals or shirts.
The validation set and the test set are also split this way, but without restricting the number of images.

We will train a model on set A (classification task with 8 classes), and try to reuse it to tackle set B (binary classification). We hope to transfer a little bit of knowledge from task A to task B, since classes in set A (sneakers, ankle boots, coats, t-shirts, etc.) are somewhat similar to classes in set B (sandals and shirts). However, since we are using Dense layers, only patterns that occur at the same location can be reused (in contrast, convolutional layers will transfer much better, since learned patterns can be detected anywhere on the image).

In [3]:

def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [4]:
X_train_A.shape

(43986, 28, 28)

In [5]:
X_train_B.shape

(200, 28, 28)

In [6]:
y_train_A[:30]

array([4, 0, 5, 7, 7, 7, 4, 4, 3, 4, 0, 1, 6, 3, 4, 3, 2, 6, 5, 3, 4, 5,
       1, 3, 4, 2, 0, 6, 7, 1], dtype=uint8)

In [7]:
y_train_B[:30]

array([1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.], dtype=float32)

In [8]:
tf.random.set_seed(42)
np.random.seed(42)

In [13]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [14]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [15]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.3455 - accuracy: 0.8785 - val_loss: 0.2868 - val_accuracy: 0.8929
Epoch 2/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.2624 - accuracy: 0.9067 - val_loss: 0.2735 - val_accuracy: 0.9018
Epoch 3/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.2397 - accuracy: 0.9148 - val_loss: 0.2466 - val_accuracy: 0.9126
Epoch 4/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.2232 - accuracy: 0.9196 - val_loss: 0.2360 - val_accuracy: 0.9158
Epoch 5/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.2143 - accuracy: 0.9237 - val_loss: 0.2244 - val_accuracy: 0.9253
Epoch 6/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.2028 - accuracy: 0.9263 - val_loss: 0.2030 - val_accuracy: 0.9292
Epoch 7/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.1910 - accuracy: 0.9312 - val_loss: 0.2233 - val_accuracy:

In [16]:
model_A.save("my_model_A.h5")

In [17]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [19]:
model_B.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [20]:
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 0s 21ms/step - loss: 0.2356 - accuracy: 0.8750 - val_loss: 0.0663 - val_accuracy: 0.9838
Epoch 2/20
7/7 [==============================] - 0s 12ms/step - loss: 0.0161 - accuracy: 0.9950 - val_loss: 0.1111 - val_accuracy: 0.9777
Epoch 3/20
7/7 [==============================] - 0s 10ms/step - loss: 0.0038 - accuracy: 1.0000 - val_loss: 0.1323 - val_accuracy: 0.9757
Epoch 4/20
7/7 [==============================] - 0s 9ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.1139 - val_accuracy: 0.9807
Epoch 5/20
7/7 [==============================] - 0s 10ms/step - loss: 4.6539e-04 - accuracy: 1.0000 - val_loss: 0.1032 - val_accuracy: 0.9817
Epoch 6/20
7/7 [==============================] - 0s 12ms/step - loss: 2.9622e-04 - accuracy: 1.0000 - val_loss: 0.0985 - val_accuracy: 0.9848
Epoch 7/20
7/7 [==============================] - 0s 13ms/step - loss: 2.2314e-04 - accuracy: 1.0000 - val_loss: 0.0970 - val_accuracy: 0.9858
Epoch 8/20
7/7 [

In [22]:
model_B.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 300)               235500    
_________________________________________________________________
dense_13 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_14 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_15 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_16 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_17 (Dense)             (None, 1)                

In [27]:
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [28]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [29]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer="adam",
                     metrics=["accuracy"])

In [30]:

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer="adam",
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 0s 21ms/step - loss: 1.2804 - accuracy: 0.7150 - val_loss: 1.0422 - val_accuracy: 0.7323
Epoch 2/4
7/7 [==============================] - 0s 10ms/step - loss: 1.0990 - accuracy: 0.7200 - val_loss: 0.8851 - val_accuracy: 0.7525
Epoch 3/4
7/7 [==============================] - 0s 10ms/step - loss: 0.9321 - accuracy: 0.7350 - val_loss: 0.7503 - val_accuracy: 0.7718
Epoch 4/4
7/7 [==============================] - 0s 10ms/step - loss: 0.8045 - accuracy: 0.7500 - val_loss: 0.6347 - val_accuracy: 0.8012
Epoch 1/16
7/7 [==============================] - 0s 20ms/step - loss: 0.2839 - accuracy: 0.8950 - val_loss: 0.0795 - val_accuracy: 0.9746
Epoch 2/16
7/7 [==============================] - 0s 11ms/step - loss: 0.0128 - accuracy: 0.9950 - val_loss: 0.0390 - val_accuracy: 0.9838
Epoch 3/16
7/7 [==============================] - 0s 10ms/step - loss: 0.0047 - accuracy: 1.0000 - val_loss: 0.0239 - val_accuracy: 0.9899
Epoch 4/16
7/7 [===============

In [31]:
model_B.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 1ms/step - loss: 0.0471 - accuracy: 0.9915


[0.04706811532378197, 0.9915000200271606]

In [32]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 1ms/step - loss: 0.0194 - accuracy: 0.9935


[0.019431374967098236, 0.9934999942779541]